In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [9]:
def asignarPV(name, dic):
    return dic[name]

In [10]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [11]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [12]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [13]:
p = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

# Separamos los datos

In [14]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'place_value']],p[['price_aprox_usd']],test_size=0.2)


# Intento hacer una pasada con algunos parametros random, estimador Gradient boosting

In [16]:
gb11 = GradientBoostingRegressor(criterion='mse', loss= 'ls', min_samples_leaf=2, max_features = 'auto') 

gb12 = GradientBoostingRegressor(criterion='mse', loss= 'ls',min_samples_leaf=3, max_features = 'auto') 

In [17]:
ada = AdaBoostRegressor(base_estimator = gb11)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

In [18]:
error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

101290922781.48845

In [19]:
ada = AdaBoostRegressor(base_estimator = gb12)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

In [20]:
error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

78093192294.942093

In [23]:
gb21 = GradientBoostingRegressor(criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=4, max_features = 'auto') 
gb22 = GradientBoostingRegressor(criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=6, max_features = 'auto') 
gb23 = GradientBoostingRegressor(criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb24 = GradientBoostingRegressor(criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=10, max_features = 'auto') 

In [25]:
ada = AdaBoostRegressor(base_estimator = gb21)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

100032499528.2256

In [26]:
ada = AdaBoostRegressor(base_estimator = gb22)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

94324662055.620743

In [28]:
ada = AdaBoostRegressor(base_estimator = gb23)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

92511503712.689545

In [29]:
ada = AdaBoostRegressor(base_estimator = gb24)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

97860966519.577118

In [30]:
gb31 = GradientBoostingRegressor(n_estimators = 1,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb32 = GradientBoostingRegressor(n_estimators = 10,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb33 = GradientBoostingRegressor(n_estimators = 50,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb34 = GradientBoostingRegressor(n_estimators = 60,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb35 = GradientBoostingRegressor(n_estimators = 70,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb36 = GradientBoostingRegressor(n_estimators = 80,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb37 = GradientBoostingRegressor(n_estimators = 90,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb38 = GradientBoostingRegressor(n_estimators = 100,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb39 = GradientBoostingRegressor(n_estimators = 110,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb310 = GradientBoostingRegressor(n_estimators = 120,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb311 = GradientBoostingRegressor(n_estimators = 150,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 
gb312 = GradientBoostingRegressor(n_estimators = 200,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto') 


In [31]:
ada = AdaBoostRegressor(base_estimator = gb31)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

136585848179.98843

In [32]:
ada = AdaBoostRegressor(base_estimator = gb32)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

97031112934.142059

In [33]:
ada = AdaBoostRegressor(base_estimator = gb33)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

78310836185.450775

In [34]:
ada = AdaBoostRegressor(base_estimator = gb34)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

75737108139.042313

In [35]:
ada = AdaBoostRegressor(base_estimator = gb35)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

75420487676.632812

In [36]:
ada = AdaBoostRegressor(base_estimator = gb36)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

75754783879.762634

In [37]:
ada = AdaBoostRegressor(base_estimator = gb37)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

88601780904.564911

In [38]:
ada = AdaBoostRegressor(base_estimator = gb38)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

100433291320.55852

In [39]:
ada = AdaBoostRegressor(base_estimator = gb39)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

72046384201.303406

In [40]:
ada = AdaBoostRegressor(base_estimator = gb310)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

78964124431.029755

In [41]:
ada = AdaBoostRegressor(base_estimator = gb311)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

80483213560.756699

In [42]:
ada = AdaBoostRegressor(base_estimator = gb312)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

106857306604.834

Elijo GradientBoostingRegressor(n_estimators = 110,criterion='mse', loss= 'ls',min_samples_leaf=3,min_samples_split=8, max_features = 'auto')

# Hago algo similar con Decision Tree

In [47]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=8, max_features = 'auto')
ada = AdaBoostRegressor(base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

72722002058.188278

In [48]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=10, max_features = 'auto')
ada = AdaBoostRegressor(base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

79291494515.168686

In [49]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=6, max_features = 'auto')
ada = AdaBoostRegressor(base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

76266063213.419785

In [50]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=2,min_samples_split=8, max_features = 'auto')
ada = AdaBoostRegressor(base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

77107453609.063004